In [1]:
'''
Movie and Book Chatbot

Keeps track of your movies and books, their ratings, and outputs recommendations based on NLP
'''

import nltk
import re
import pickle
from datetime import datetime
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from collections import defaultdict

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
def interact_with_chatbot(assistant, user_id):
    print("Welcome to the Personal Book and Movie chatbot!")
    print("Type 'quit' to exit.")

    while True:
        user_input = input(f"{user_id}: ")
        if user_input.lower() == 'quit':
            break

        response = assistant.respond(user_id, user_input)
        print(f"Assistant: {response}")

    assistant.save_user_data()
    print("Goodbye!")

In [3]:
class UserModel:
    def __init__(self, name):
        self.name = name
        self.info = {}
        self.likes = {}
        self.dislikes = {}
        self.events = []

In [4]:
class PersonalAssistant:
    def __init__(self, user_data_file):
        self.user_data_file = user_data_file
        self.users = self.load_user_data()
    
    def load_user_data(self):
        try:
            with open(self.user_data_file, 'rb') as f:
                return pickle.load(f)
        except (FileNotFoundError, EOFError):
            return {}

    def save_user_data(self):
        with open(self.user_data_file, 'wb') as f:
            pickle.dump(self.users, f)

    def get_user_model(self, user_id):
        if user_id not in self.users:
            self.users[user_id] = UserModel("unknown")
        return self.users[user_id]
    
    def update_user_model(self, user_model, text):
        
        if "name" in text.lower():
            tagged_text = pos_tag(word_tokenize(text))
            for word, pos in tagged_text:
                if pos == 'NNP':
                    user_model.name = word.capitalize()

        likes = re.findall(r'I like (\w+)', text, re.IGNORECASE)
        for like in likes:
            category, item = self.get_item_category(like)
            user_model.likes.setdefault(category, []).append(item)

        dislikes = re.findall(r'I dislike (\w+)', text, re.IGNORECASE)
        for dislike in dislikes:
            category, item = self.get_item_category(dislike)
            user_model.dislikes.setdefault(category, []).append(item)
            
    def get_item_category(self, item):
        if item.lower() in ["book", "movie"]:
            return "entertainment", item.lower()
        return "other", item.lower()

    def respond(self, user_id, text):
        user_model = self.get_user_model(user_id)
        self.update_user_model(user_model, text)
        
        if 'what is my name' in text.lower():
            return f"Your name is {user_model.name}"        

        if user_model.name is 'unknown':
            return f"Hello!, I've noticed I don't know your name in my knowledge base. Could you provide it so I know how to address you?"
            

            
        if any(user_input in text for user_input in ['hello', 'hi', 'greetings']):
            return f"Hello, {user_model.name}! How can I help you today?" 
         
        if "name" in text.lower():
            return f"Hello, {user_model.name}! How can I help you today?"

        if "like" in text.lower() or "dislike" in text.lower():
            return f"Got it, {user_model.name}. I've updated your preferences."

        if "favorite" in text.lower():
            category = re.search(r'favorite (\w+)', text, re.IGNORECASE)
            if category and category.group(1).lower() in user_model.likes:
                items = ', '.join(user_model.likes[category.group(1).lower()])
                return f"{user_model.name}, your favorite {category.group(1).lower()} are: {items}"
            return f"I couldn't find any favorite {category.group(1).lower()} for you, {user_model.name}."

        return f"I'm not sure what you're asking, {user_model.name}. Could you please rephrase?"

<>:52: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:52: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\myaku\AppData\Local\Temp\ipykernel_7752\4203032345.py:52: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if user_model.name is 'unknown':


In [ ]:
user_id = "Matthew Y"
user_data_file = user_id + '.pkl'
assistant = PersonalAssistant(user_data_file)

interact_with_chatbot(assistant, user_id)

Welcome to the Personal Book and Movie chatbot!
Type 'quit' to exit.
Matthew Y: my name is Joe
Assistant: Hello, Joe! How can I help you today?
Matthew Y: King arthur went on a quest
Assistant: I'm not sure what you're asking, Joe. Could you please rephrase?
Matthew Y: my name is Matt
Assistant: Hello, Matt! How can I help you today?
Matthew Y: change my name to Cathryn
Assistant: Hello, Matt! How can I help you today?
Matthew Y: change my name to cade
Assistant: Hello, Matt! How can I help you today?
Matthew Y: change my name to Ana
Assistant: Hello, Matt! How can I help you today?
Matthew Y: name Ana
Assistant: Hello, Ana! How can I help you today?
